In [1]:
from mlops.project_libraries import FastAPI, mlflow, os, np
from mlops.util_funcs import scrub_data, PredictionRequest

In [2]:
# fill in AWS profile
os.environ["AWS_PROFILE"] = "demiga-g"

# Setting tracking uri (unique resource identifier)
TRACKING_SERVER_HOST = '127.0.0.1'  # '16.171.136.194'
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
experiment_id = 6
run_id = '45a3990ff1e140afbe48334a8422bec7'

In [ ]:
def load_model():
    """Load a model from an experiment run."""
    base_location = 'mlflow-artifacts:'  # 's3://midega-mlflow-artifacts'
    logged_model = f"{base_location}/{experiment_id}/{run_id}/artifacts/model"
    return mlflow.pyfunc.load_model(logged_model)


In [ ]:
def predict_responses(features):
    model = load_model()
    y_pred = model.predict(features)
    return y_pred

In [ ]:
app = FastAPI('customer-response')

@app.get('/')
def root_page():
    """
    This function serves as the root endpoint for the FastAPI application.
    It returns a welcome message when accessed via a GET request.

    Parameters:
    None

    Returns:
    dict: A dictionary containing a welcome message.
    """
    return {"message": "Welcome to the Customer Response Prediction API"}

In [ ]:
@app.post('/predictions')
def predict_endpoint(request: PredictionRequest):
    details = request.dict()
    # features = pd.DataFrame([details])
    cleaned_features = scrub_data(details)
    preds = predict_responses(cleaned_features)
    result = {'response': preds.tolist()}
    return result